# 모델에 필요한 파일 받기

In [ ]:
#현재 위치 확인
!pwd

/content


In [ ]:
#ACGPN 모델 파일 다운로드
!git clone https://github.com/kairess/ACGPN.git

Cloning into 'ACGPN'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 165 (delta 21), reused 19 (delta 18), pack-reused 141
Receiving objects: 100% (165/165), 303.15 KiB | 21.65 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [ ]:
#다운로드 받은 파일의 위치로 이동
%cd ACGPN

/content/ACGPN


# dependencies 및 사전 파일 준비

In [ ]:
!pip install -U --no-cache-dir gdown --pre -qq
!pip install ninja -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 10.7 MB/s eta 0:00:00


In [ ]:
import gdown
import numpy as np
from PIL import Image
import IPython
import os
import sys
import time

from predict_pose import generate_pose_keypoints

In [ ]:
#모델을 돌리기 위한 파일 준비
!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_colormask
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir Data_preprocessing/test_label
!mkdir Data_preprocessing/test_mask
!mkdir Data_preprocessing/test_pose
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

In [ ]:
#현재 위치 확인
!pwd

/content/ACGPN


In [ ]:
#휴먼 세그멘테이션 모델 파일 다운로드
!git clone https://github.com/levindabhi/Self-Correction-Human-Parsing-for-ACGPN.git
#옷 마스킹 모델 파일 다운로드
!git clone https://github.com/levindabhi/U-2-Net.git

Cloning into 'Self-Correction-Human-Parsing-for-ACGPN'...
remote: Enumerating objects: 769, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 769 (delta 72), reused 62 (delta 62), pack-reused 658
Receiving objects: 100% (769/769), 3.80 MiB | 22.22 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Cloning into 'U-2-Net'...
remote: Enumerating objects: 822, done.
remote: Counting objects: 100% (822/822), done.
remote: Compressing objects: 100% (439/439), done.
remote: Total 822 (delta 389), reused 793 (delta 379), pack-reused 0
Receiving objects: 100% (822/822), 30.71 MiB | 7.59 MiB/s, done.
Resolving deltas: 100% (389/389), done.


# 사전학습된 모델 다운로드

# 포즈 예측 모델

In [ ]:
!gdown 1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko -O pose/pose_iter_440000.caffemodel

Downloading...
From (uriginal): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko
From (redirected): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko&confirm=t&uuid=0c7afce5-e23d-4402-9180-bd67bd05ef7c
To: /content/ACGPN/pose/pose_iter_440000.caffemodel
100% 209M/209M [00:05<00:00, 38.2MB/s]


# 휴먼 세그멘테이션 모델

In [ ]:
gdown.download('https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH', 'lip_final.pth', quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=cf16fd5d-a07b-4d84-9328-39255208cd93
To: /content/ACGPN/lip_final.pth
100%|██████████| 267M/267M [00:02<00:00, 89.8MB/s]


'lip_final.pth'

# U-2-Net 모델 (옷 마스크 추출 모델)

In [ ]:
#U-2-Net 모델을 돌릴 때 필요한 파일 준비를 위해 해당 모델 파일로 이동
%cd U-2-Net

#U-2-Net 모델에 필요한 파일 생성
!mkdir saved_models
!mkdir saved_models/u2net
!mkdir saved_models/u2netp

#사전 학습된 U-2-Net 모델 다운로드
!gdown 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O saved_models/u2netp/u2netp.pth
!gdown 1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O saved_models/u2net/u2net.pth

import u2net_load
import u2net_run

#로드하려면 쿠다 필요 (위치 확인용)
u2net = u2net_load.model(model_name='u2netp')

#U-2-Net 모델을 위한 파일 준비가 끝나면 부모 파일로 이동
%cd ..

/content/ACGPN/U-2-Net
mkdir: cannot create directory ‘saved_models’: File exists
Downloading...
From: https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy
To: /content/ACGPN/U-2-Net/saved_models/u2netp/u2netp.pth
100% 4.68M/4.68M [00:00<00:00, 18.4MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ
From (redirected): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ&confirm=t&uuid=86ae5e25-bc28-4fe5-a55f-bed8f7817ddc
To: /content/ACGPN/U-2-Net/saved_models/u2net/u2net.pth
100% 176M/176M [00:04<00:00, 38.0MB/s]
...load U2NEP---4.7 MB
/content/ACGPN


# ACGPN모델 (사진 합성 모델)

In [ ]:
#ACGPN 모델에 필요한 파일 생성
!mkdir checkpoints

#사전 학습된 ACGPN 모델 다운로드
gdown.download('https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx', output='checkpoints/ACGPN_checkpoints.zip', quiet=False)

#압축된 모델 파일 풀어서 전에 만든 파일위치에 넣어줌
!unzip checkpoints/ACGPN_checkpoints.zip -d checkpoints

Downloading...
From (uriginal): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx
From (redirected): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx&confirm=t&uuid=c50cec54-2d99-4921-8251-453ad2279bfb
To: /content/ACGPN/checkpoints/ACGPN_checkpoints.zip
100%|██████████| 524M/524M [00:09<00:00, 58.0MB/s]


Archive:  checkpoints/ACGPN_checkpoints.zip
   creating: checkpoints/label2city/
  inflating: checkpoints/label2city/latest_net_G.pth  
  inflating: checkpoints/label2city/latest_net_G1.pth  
  inflating: checkpoints/label2city/latest_net_G2.pth  
  inflating: checkpoints/label2city/latest_net_U.pth  
  inflating: checkpoints/label2city/opt.txt  


In [ ]:
!pwd

/content/ACGPN


In [ ]:
!pip install colabcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 56.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 

In [ ]:
!pip install fastapi["all"]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 7.0 MB/s eta 0:00:00
  Preparing metad

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse, JSONResponse
import os
import time
from PIL import Image


app = FastAPI()

가설 2 : 전처리부터 추론까지의 코드를 합쳐서 api로 만든다

In [ ]:

# Define the paths for the input directories and models
CLOTH_DIR = "inputs/cloth"
MODEL_DIR = "inputs/img"
PREPROCESS_DIR = "Data_preprocessing"
COLOR_DIR = f"{PREPROCESS_DIR}/test_color"
IMG_DIR = f"{PREPROCESS_DIR}/test_img"
EDGE_DIR = f"{PREPROCESS_DIR}/test_edge"
LABEL_DIR = f"{PREPROCESS_DIR}/test_label"
POSE_DIR = f"{PREPROCESS_DIR}/test_pose"
MODEL_RESTORE_PATH = "lip_final.pth"

#for test
@app.get("/")
def home():
  return 'hello this is model'

#옷 데이터 넣어주기
@app.post("/inputs/cloth")
async def upload_photo(file: UploadFile):

    UPLOAD_DIR = "./inputs/cloth"  
    
    content = await file.read()
    # 저장할 이미지 이름을 시간으로 변경
    filename = f"{int(time.time())}.png"  
    # 서버 로컬 스토리지에 이미지 저장 (쓰기)
    with open(os.path.join(UPLOAD_DIR, filename), "wb") as fp:
        fp.write(content)  
 
    return {"filename": filename}

#모델 데이터 넣어주기
@app.post("/inputs/img")
async def upload_photo(file: UploadFile):

    UPLOAD_DIR = "./inputs/img"  
    
    content = await file.read()
    # 저장할 이미지 이름을 시간으로 변경
    filename = f"{int(time.time())}.png"  
    # 서버 로컬 스토리지에 이미지 저장 (쓰기)
    with open(os.path.join(UPLOAD_DIR, filename), "wb") as fp:
        fp.write(content)  
 
    return {"filename": filename}

@app.get("/predictimage")
async def predict():
    sorted_cloth_files = sorted(os.listdir('inputs/cloth'))
    
    cloth_name = f'cloth_{int(time.time())}.png'

    cloth_path = os.path.join('inputs/cloth', sorted_cloth_files[0])

    cloth = Image.open(cloth_path)

    cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')

    cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name))

    u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

      #모델 데이터셋이 올바른 위치에 있는지 확인
    sorted_img_files = sorted(os.listdir('inputs/img'))

    #전처리 할 모델 데이터 이름을 시간으로 지정
    img_name = f'img_{int(time.time())}.png'

    #전처리 할 모델 데이터의 파일 위치 확인
    img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])

    #이미지 할당
    img = Image.open(img_path)

    #이미지 사이즈 조절
    img = img.resize((192,256), Image.BICUBIC)

    #전처리를 위해 모델 이미지 파일의 위치값을 해당 파일 위치에 지정해준 모델 데이터 이름으로 지정
    img_path = os.path.join('Data_preprocessing/test_img', img_name)

    #지정해준 모델 이미지 파일의 위치값을 바탕으로 이미지 저장
    img.save(img_path)

    #전처리를 위해 휴먼 세그멘테이션 모델을 parser를 이용하여 돌림 ***parser는 데이터셋의 종류, 사전학습 모델 파일의 위치, 전처리 할 모델 이미지 위치, 전처리 후 이미지 저장할 위치
    !python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py --dataset 'lip' --model-restore 'lip_final.pth' --input-dir 'Data_preprocessing/test_img' --output-dir 'Data_preprocessing/test_label'

    #모델 포즈 예측 모델을 돌리고 저장할 파일 위치와 파일 이름 형식을 지정
    pose_path = os.path.join('Data_preprocessing/test_pose', img_name.replace('.png', '_keypoints.json'))

    #모델 포즈 예측 모델을 전처리 할 모델 이미지 위치, 전처리 후 파일 위치를 받아서 돌리고 저장
    generate_pose_keypoints(img_path, pose_path)
    
    with open('Data_preprocessing/test_pairs.txt', 'w') as f:
          f.write(f'{img_name} {cloth_name}')

    
    !python3 test.py

    result_path = os.path.join('test', 'final', f'{img_name.split(".")[0]}_final.png')
    with open(result_path, "rb") as f:
      result_image = f.read()

    return result_image 

In [ ]:
from colabcode import ColabCode
cc = ColabCode(port = 12000, code = False)

In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://81f7-34-143-161-215.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [159]
INFO:uvicorn.error:Started server process [159]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Generating mask for: cloth_1682036953.png
Saving output at Data_preprocessing/test_edge/cloth_1682036953.png
Generating mask for: cloth_1682036754.png
Saving output at Data_preprocessing/test_edge/cloth_1682036754.png
100% 2/2 [00:00<00:00,  4.37it/s]
File saved at Data_preprocessing/test_pose/img_1682036954_keypoints.json
INFO:     118.223.10.215:0 - "GET /predictimage HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 396, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.9/dist-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.9/dist-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.9/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.9/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.9/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.9/dist-packag

# 가설 1: 전처리부터 추론까지의 코드는 api안에 넣어준다. => 실패

In [ ]:
!pwd

/content/ACGPN


In [ ]:
!pip install colabcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 51.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 M

In [ ]:
!pip install fastapi["all"]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 223.5 kB/s eta 0:00:00
  Preparing metad

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse, JSONResponse
import os
import time
from PIL import Image


app = FastAPI()

In [ ]:

# Define the paths for the input directories and models
CLOTH_DIR = "inputs/cloth"
MODEL_DIR = "inputs/img"
PREPROCESS_DIR = "Data_preprocessing"
COLOR_DIR = f"{PREPROCESS_DIR}/test_color"
IMG_DIR = f"{PREPROCESS_DIR}/test_img"
EDGE_DIR = f"{PREPROCESS_DIR}/test_edge"
LABEL_DIR = f"{PREPROCESS_DIR}/test_label"
POSE_DIR = f"{PREPROCESS_DIR}/test_pose"
MODEL_RESTORE_PATH = "lip_final.pth"

#for test
@app.get("/")
def home():
  return 'hello this is model'

#옷 데이터 넣어주기
@app.post("/inputs/cloth")
async def upload_photo(file: UploadFile):

    UPLOAD_DIR = "./inputs/cloth"  
    
    content = await file.read()
    # 저장할 이미지 이름을 시간으로 변경
    filename = f"{int(time.time())}.png"  
    # 서버 로컬 스토리지에 이미지 저장 (쓰기)
    with open(os.path.join(UPLOAD_DIR, filename), "wb") as fp:
        fp.write(content)  
 
    return {"filename": filename}

#모델 데이터 넣어주기
@app.post("/inputs/img")
async def upload_photo(file: UploadFile):

    UPLOAD_DIR = "./inputs/img"  
    
    content = await file.read()
    # 저장할 이미지 이름을 시간으로 변경
    filename = f"{int(time.time())}.png"  
    # 서버 로컬 스토리지에 이미지 저장 (쓰기)
    with open(os.path.join(UPLOAD_DIR, filename), "wb") as fp:
        fp.write(content)  
 
    return {"filename": filename}

@app.get("/preprocess_cloth")
async def preprocess_cloth():
    sorted_cloth_files = sorted(os.listdir('inputs/cloth'))
    
    cloth_name = f'cloth_{int(time.time())}.png'

    cloth_path = os.path.join('inputs/cloth', sorted_cloth_files[0])

    cloth = Image.open(cloth_path)

    cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')

    cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name))

    u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

    return cloth_name


In [ ]:
#위에 적어놓음
#from fastapi.responses import JSONResponse


###################################################
#옷, 모델 데이터 넣어주기 + u2net모델 성공 but 모델은 안돌아감#
###################################################


@app.get("/preprocess_img")
async def preprocess_img():

    #모델 데이터셋이 올바른 위치에 있는지 확인
    sorted_img_files = sorted(os.listdir('inputs/img'))

    #전처리 할 모델 데이터 이름을 시간으로 지정
    img_name = f'img_{int(time.time())}.png'

    #전처리 할 모델 데이터의 파일 위치 확인
    img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])

    #이미지 할당
    img = Image.open(img_path)

    #이미지 사이즈 조절
    img = img.resize((192,256), Image.BICUBIC)

    #전처리를 위해 모델 이미지 파일의 위치값을 해당 파일 위치에 지정해준 모델 데이터 이름으로 지정
    img_path = os.path.join('Data_preprocessing/test_img', img_name)

    #지정해준 모델 이미지 파일의 위치값을 바탕으로 이미지 저장
    img.save(img_path)

    #전처리를 위해 휴먼 세그멘테이션 모델을 parser를 이용하여 돌림 ***parser는 데이터셋의 종류, 사전학습 모델 파일의 위치, 전처리 할 모델 이미지 위치, 전처리 후 이미지 저장할 위치
    !python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py --dataset 'lip' --model-restore 'lip_final.pth' --input-dir 'Data_preprocessing/test_img' --output-dir 'Data_preprocessing/test_label'

    #모델 포즈 예측 모델을 돌리고 저장할 파일 위치와 파일 이름 형식을 지정
    pose_path = os.path.join('Data_preprocessing/test_pose', img_name.replace('.png', '_keypoints.json'))

    #모델 포즈 예측 모델을 전처리 할 모델 이미지 위치, 전처리 후 파일 위치를 받아서 돌리고 저장
    generate_pose_keypoints(img_path, pose_path)
    return img_name

cloth_name = preprocess_cloth()
img_name = preprocess_img()

In [ ]:
!pwd

/content/ACGPN


In [ ]:
### 현재 에러 메시지
# cloth_name과 img_name 파라미터가 필요한데 없어서 422error가 발생한 것이라고 함

# {
#     "detail": [
#         {
#             "loc": [
#                 "query",
#                 "cloth_name"
#             ],
#             "msg": "field required",
#             "type": "value_error.missing"
#         },
#         {
#             "loc": [
#                 "query",
#                 "img_name"
#             ],
#             "msg": "field required",
#             "type": "value_error.missing"
#         }
#     ]
# }

####################

# @app.get("/predict")
# async def predict_image():
#      with open('Data_preprocessing/test_pairs.txt', 'w') as f:
#           f.write(f'{img_name} {cloth_name}')

#      %cd  
#      os.system(f"python3 test.py")

#      result_path = os.path.join('test', 'final', f'{img_name.split(".")[0]}_final.png')
#      with open(result_path, "rb") as f:
#         result_image = f.read()

#      return result_image 

####################

@app.get("/predict")
async def predict_image(cloth_name: str, img_name: str):
    cloth_name = preprocess_cloth(cloth_name)
    img_name = preprocess_img(img_name)
    
    with open('Data_preprocessing/test_pairs.txt', 'w') as f:
        f.write(f'{img_name} {cloth_name}')

    os.system(f"python3 test.py")

    result_path = os.path.join('test', 'final', f'{img_name.split(".")[0]}_final.png')
    with open(result_path, "rb") as f:
        result_image = f.read()

    return result_image

In [ ]:
from colabcode import ColabCode
cc = ColabCode(port = 12000, code = False)

In [ ]:
cc.run_app(app=app)

INFO:     Started server process [325]
INFO:uvicorn.error:Started server process [325]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://5bb6-34-83-93-81.ngrok.io" -> "http://localhost:12000"
INFO:     118.223.10.215:0 - "POST /inputs/img HTTP/1.1" 200 OK
INFO:     118.223.10.215:0 - "POST /inputs/cloth HTTP/1.1" 200 OK
Generating mask for: cloth_1682025119.png
Saving output at Data_preprocessing/test_edge/cloth_1682025119.png
INFO:     118.223.10.215:0 - "GET /preprocess_cloth HTTP/1.1" 200 OK
100% 1/1 [00:00<00:00,  1.86it/s]
File saved at Data_preprocessing/test_pose/img_1682025136_keypoints.json
INFO:     118.223.10.215:0 - "GET /preprocess_img HTTP/1.1" 200 OK
INFO:     118.223.10.215:0 - "GET /predict HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [325]
INFO:uvicorn.error:Finished server process [325]
